## MNIST 손글씨 -CNN v2
  - 2 x Conv2D layers
  - 1 MaxPooling2D layer
  - 1 1 FCN layer
  - 1 Output layer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
tf.__version__

'2.14.0'

- 데이터 전처리

In [3]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

11490434/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [4]:
# X data의 차원 변경 및 정규화(0 ~ 1)
X_train = X_train.reshape(-1, 28, 28, 1) / 255.
X_test = X_test.reshape(-1, 28, 28, 1) / 255.


In [5]:
# Y_data의 one-hot-encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_train.shape, Y_test.shape

((60000, 10), (10000, 10))

- 모델 정의/설정/학습/평가

In [6]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [7]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu' ),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),                     #  MaxPooling2D(pool_size(2,2))
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')

])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 64)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 512)               4719104   
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                        

In [8]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [9]:
!mkdir models
!ls -l

total 8
drwxr-xr-x 2 root root 4096 Nov  8 02:05 models
drwxr-xr-x 1 root root 4096 Nov  6 14:20 sample_data


In [10]:
model_path = 'models/best-mnist-v2.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [11]:
hist = model.fit(
    X_train, Y_train, validation_split=0.2, epochs=100, batch_size=256,
    callbacks=[mc,es]
)

Epoch 1/100
188/188 [==============================] - ETA: 0s - loss: 0.1915 - accuracy: 0.9424
Epoch 1: val_loss improved from inf to 0.05716, saving model to models/best-mnist-v2.h5
188/188 [==============================] - 13s 17ms/step - loss: 0.1915 - accuracy: 0.9424 - val_loss: 0.0572 - val_accuracy: 0.9835
Epoch 2/100
  6/188 [..............................] - ETA: 2s - loss: 0.0482 - accuracy: 0.9824

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


186/188 [============================>.] - ETA: 0s - loss: 0.0470 - accuracy: 0.9854
Epoch 2: val_loss improved from 0.05716 to 0.04292, saving model to models/best-mnist-v2.h5
188/188 [==============================] - 3s 15ms/step - loss: 0.0469 - accuracy: 0.9854 - val_loss: 0.0429 - val_accuracy: 0.9869
Epoch 3/100
184/188 [============================>.] - ETA: 0s - loss: 0.0268 - accuracy: 0.9916
Epoch 3: val_loss did not improve from 0.04292
188/188 [==============================] - 3s 14ms/step - loss: 0.0271 - accuracy: 0.9915 - val_loss: 0.0430 - val_accuracy: 0.9877
Epoch 4/100
186/188 [============================>.] - ETA: 0s - loss: 0.0181 - accuracy: 0.9942
Epoch 4: val_loss improved from 0.04292 to 0.03904, saving model to models/best-mnist-v2.h5
188/188 [==============================] - 3s 16ms/step - loss: 0.0180 - accuracy: 0.9942 - val_loss: 0.0390 - val_accuracy: 0.9884
Epoch 5/100
186/188 [============================>.] - ETA: 0s - loss: 0.0103 - accuracy: 0.99

In [12]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0341 - accuracy: 0.9893


[0.03412098437547684, 0.989300012588501]

- 학습과정 시각화

In [13]:
y_acc = hist.history['accuracy']
y_vloss = hist.history['val_loss']
xs = np.arange(1,len(y_acc)+1)


In [1]:
plt.figure(figsize=(12,6))
plt.plot(xs, y_acc, label='train accuracy')
plt.plot(xs, y_vloss, label='validation loss')
plt.legend(), plt.grid(), plt.xlabel('epoch')
plt.title('MNIST CNN Model v2')
plt.show()

NameError: name 'plt' is not defined